In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileSimradRaw,FileSimradRaw_mapped,SimradDatagram_type_from_string,datagram_type_to_string,t_SimradDatagramIdentifier
import themachinethatgoesping.echosounders.simrad as simrad
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import themachinethatgoesping.tools as tools
import themachinethatgoesping.navigation.nmea_0183 as NMEA
import numpy
import os, sys
from time import time
from tqdm import tqdm
from collections import defaultdict
import numpy as np

#load files
folders=[]
with open('test_data.txt','r') as ifi:
    for line in ifi:
        print("Folder:", line)
        folders.append(line)

if not len(folders):
    print("Please add test_data folders to 'test_data.txt'")

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [ ]:
fm = FileSimradRaw_mapped(files)
print(fm)

In [ ]:
#%%timeit -n1 -r1
fail = 0
for ping in tqdm(fm.pings()):
    ping.get_sv(dB = True)

In [ ]:
pings = fm.pings()[::100]
#pings = fm.pings()[280000:290000:1]

nsamples = min(7000,pings.max_number_of_samples())
#nsamples = 7000

image = np.empty((len(pings),nsamples),dtype=np.float32)
image.fill(np.nan)

for i,ping in enumerate(tqdm(pings)):
    sv = ping.get_sv_stacked(dB = True)
    #sv = ping.sample_data.get_power(dB = True)
    ns = min(nsamples,len(sv))
    image[i,:ns] = sv[:ns]
    

In [ ]:
plt.close("echo")
fig = plt.figure("echo")
ax = fig.subplots()
ax.imshow(image.transpose(), aspect = 'auto', extent = [0,len(fm.pings()),7000,0])